In [ ]:
%pip install git+https://github.com/huggingface/transformers.git
%pip install Pillow redvid
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import torch

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)


/home/lettuce/miniconda3/envs/trainingLlama2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/lettuce/miniconda3/envs/trainingLlama2/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [01:28<00:00, 44.42s/it]


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [33]:
import requests
from redvid import Downloader
import subprocess
from PIL import Image


def isMediaDomain(url):
  for mediaDomain in mediaDomains.keys():
    if mediaDomain in url or url.startswith('self.'):
      return True
  return False

video_input_file = 'video.mp4'


img_output_file = 'image.jpg'
video_output_file = img_output_file#'video.jpg'

def downloadRedditVideo(url):
  # delete video.mp4 if it exists
  subprocess.call(['rm', '-f', video_input_file])
  reddit = Downloader(max_q=True)
  reddit.log = False
  reddit.url = url
  reddit.path = "./"
  reddit.filename = video_input_file
  reddit.download()
  print("Downloaded " + video_input_file)

  image = extractFrameFromVideo()

  image = Image.open(video_output_file)
  return image

def extractFrameFromVideo():
  input_file = video_input_file
  output_file = video_output_file

  #delete video.jpg if it exists
  subprocess.call(['rm', '-f', output_file])

  duration = float(subprocess.check_output(['ffprobe', '-i', input_file, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=%s' % ("p=0")]).strip())
  middle_time = duration / 2
  #extract the frame
  subprocess.call(['ffmpeg', '-i', input_file, '-ss', str(middle_time), '-vframes', '1', '-q:v', '2', output_file], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
  print("Extracted frame from " + input_file + " to " + output_file)

  image = Image.open(output_file)
  return image


def generate_blip2(image, context=None):
  if context:
    inputs = processor(images=image, text=context, return_tensors="pt").to(device, torch.float16)
  else:
    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
  generated_ids = model.generate(**inputs, max_new_tokens=100)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
  return generated_text

def downloadVideo(url):
  if 'reddit' in url:
    return downloadRedditVideo(url)
  else:
    video_data = requests.get(url).content
    with open(video_input_file, 'wb') as handler:
      handler.write(video_data)
    
    return extractFrameFromVideo()


def downloadImage(url):
  print("Downloading " + url)
  if 'gifv' in url:
    # change to gif
    url = url.replace('gifv', 'mp4')
    return downloadVideo(url)
  img_data = requests.get(url).content
  with open(img_output_file, 'wb') as handler:
      handler.write(img_data)

  image = Image.open(img_output_file)
  return image


def downloadYoutubeThumbnail(url):
  if 'youtu.be' in url:
    # after / and before ?
    youtube_id = url.split('/')[-1]
    youtube_id = youtube_id.split('?')[0]
    youtube_id = youtube_id.split('&')[0]
  else:
    youtube_id = url.split('v=')[-1]
    youtube_id = youtube_id.split('?')[0]
    youtube_id = youtube_id.split('&')[0]
  print("youtube_id", youtube_id)
  thumbnail_url = 'https://img.youtube.com/vi/' + youtube_id + '/maxresdefault.jpg'
  print("thumbnail_url", thumbnail_url)
  return downloadImage(thumbnail_url)

def downloadRedditMedia(domain, url):
  # delete image.jpg if it exists
  subprocess.call(['rm', '-f', img_output_file])
  #delete video.mp4 if it exists
  subprocess.call(['rm', '-f', video_input_file])
  if domain in mediaDomains:
    return mediaDomains[domain](url)
  else:
    return None
  
  # call the right function based on domain
  func = mediaDomains[domain]
  return func(url)

mediaDomains = {
    "i.redd.it": downloadImage,
    "i.reddituploads.com": downloadImage,
    "v.redd.it": downloadRedditVideo,
    "i.imgur.com": downloadImage,
    "youtu.be": downloadYoutubeThumbnail,
    "youtube.com": downloadYoutubeThumbnail,
  }


In [24]:
# load image.jpg
image = Image.open('image.jpg')
generate_blip2(image, "Question: What is the title of this picture? Answer: german riot police defeated and humiliated by some kind of mud wizard. Question: What is happening? Answer: ")

'~~german~~ german riot police defeated and humiliated by some kind of mud wizard.'

## load subreddits

In [34]:
import glob

json_files = glob.glob('./data/*.json')
print(json_files)

import json

def getRedditSubreddits(json_files):
  subreddits = {}
  for json_file in json_files:
    with open(json_file) as f:
      data = json.load(f)
      firstKey = next(iter(data))
      subredditName = data[firstKey]['subreddit']
      subreddits[subredditName] = data
  return subreddits

def saveredditSubreddits(subreddits):
  for subreddit in subreddits:
    with open('./data/' + subreddit + '_top_posts.json', 'w') as outfile:
      json.dump(subreddits[subreddit], outfile, indent=4)

subreddits = getRedditSubreddits(json_files)

print("subreddits", subreddits.keys())


['./data/facepalm_top_posts.json', './data/announcements_top_posts.json', './data/DunderMifflin_top_posts.json', './data/wholesomememes_top_posts.json', './data/science_top_posts.json', './data/BikiniBottomTwitter_top_posts.json', './data/freefolk_top_posts.json', './data/MadeMeSmile_top_posts.json', './data/StarWarsBattlefront_top_posts.json', './data/books_top_posts.json', './data/tifu_top_posts.json', './data/nextfuckinglevel_top_posts.json', './data/EarthPorn_top_posts.json', './data/gifs_top_posts.json', './data/AmItheAsshole_top_posts.json', './data/todayilearned_top_posts.json', './data/me_irl_top_posts.json', './data/WatchPeopleDieInside_top_posts.json', './data/thanosdidnothingwrong_top_posts.json', './data/Showerthoughts_top_posts.json', './data/Wellthatsucks_top_posts.json', './data/wallstreetbets_top_posts.json', './data/nottheonion_top_posts.json', './data/comics_top_posts.json', './data/PewdiepieSubmissions_top_posts.json', './data/awfuleverything_top_posts.json', './data

In [36]:
for subreddit_name in subreddits:
  subreddit = subreddits[subreddit_name]
  print("subreddit", subreddit_name)
  for post_id in subreddit:
    post = subreddit[post_id]
    isValidUrlPost = post['url'] != None and post['text'] == None and isMediaDomain(post['url'])
    if not isValidUrlPost:
      continue

    #print url
    print(post['url'])
    print(post['title'])
    image = downloadRedditMedia(post['domain'], post['url'])
    #display image
    #image.show()
    #generate caption
    caption = generate_blip2(image)#, "Question: What is the title of this picture? Answer: " + post['title'] + " Question: What is happening? Answer: ")
    print(caption)
    #save caption
    post['text'] = caption
    

#saveredditSubreddits(subreddits)
    

subreddit facepalm
https://i.redd.it/ypuj1e6u52f81.jpg
Math is a Myth.


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/lettuce/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


a twee with a woman and a man on it
https://i.redd.it/2hb9rutko0n61.jpg
The state of the world.


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/lettuce/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


my father in law believes a youtube video that two doctors told him vaccines change your dna how is your weekend going?
https://i.redd.it/omyb4g6wwby41.jpg
Scientific name = poison


...

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/lettuce/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


two pictures of a family with a child and a baby
https://i.redd.it/ijt1f3ditjk21.jpg
Should I call you Jimothy?


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/lettuce/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


if you take a character from the office and the michael jackson, you're gonna get a jimmy jimmy
https://i.imgur.com/KXLKw9W.jpg
Custom background for your Zoom and Teams calls. You’re welcome.


ValueError: conversion not supported